# IBM Toronto Neighborhood Clustering & Analysis

## Import Libraries

In [138]:
import json
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import geopy

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Scrape Toronto Postal Codes from Wikipedia to Pandas Dataframe

In [139]:
#Get the webpage
toronto_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto_soup = BeautifulSoup(toronto_url.content, 'html.parser')

In [140]:
#Get the table rows
neighbor_table = toronto_soup.find(class_='wikitable sortable')
table_only = neighbor_table.find('tbody')
rows = table_only.find_all('tr')

In [141]:
#test = [item.get_text().strip() for item in rows[1].find_all('td')]
#test

In [142]:
#Save column headers
columns = [row.get_text().strip() for row in rows[0].find_all('th')]
columns

['Postcode', 'Borough', 'Neighbourhood']

In [143]:
#Parse table contents
l = []
for row in rows[1:]:
    td = row.find_all('td')
    l.append([item.get_text().strip() for item in td])
    

In [532]:
#Create PD DateFrame
pd_toronto = pd.DataFrame(l, columns=columns)
pd_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Convert Pandas Dataframe to Appropriate Format

In [533]:
#Drop "Not Assigned" Boroughs
pd_toronto2 = pd_toronto.drop(pd_toronto[pd_toronto.Borough == 'Not assigned'].index).reset_index().drop(columns=['index'])

In [534]:
#Combine neighborhoods into 1 postcode as a list

#Split into neighbourhood array list by postal code and postcode & borough
#To later be merged
pd_split1 = pd_toronto2.groupby('Postcode')['Neighbourhood'].apply(np.array)
pd_split1.head()

Postcode
M1B                            [Rouge, Malvern]
M1C    [Highland Creek, Rouge Hill, Port Union]
M1E         [Guildwood, Morningside, West Hill]
M1G                                    [Woburn]
M1H                                 [Cedarbrae]
Name: Neighbourhood, dtype: object

In [535]:
#Postcode/borough split
pd_split2 = pd_toronto2[['Postcode', 'Borough']]
pd_split2.head()

,Postcode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
4,M6A,North York


In [573]:
#Merge the two together
pd_merge = (pd_split2.merge(pd_split1,
                             how='left',
                            left_on='Postcode',
                            right_index=True))

#Drop any duplicate rows
pd_merge.drop_duplicates(subset='Postcode', keep='first', inplace=True)

#Reset the index and drop the extra index column
pd_merge.reset_index(inplace=True)
pd_merge.drop(columns='index', inplace=True)

pd_merge.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,[Harbourfront]
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Downtown Toronto,[Queen's Park]


In [560]:
pd_merge.iloc[5][2]

array(['Not assigned'], dtype=object)

## Replace "Not assigned" Lists with Buroughs

In [561]:
#Change to list form
toronto = pd_merge.copy()
toronto['Neighbourhood'] = toronto['Neighbourhood'].apply(lambda x: x.tolist())
toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,[Harbourfront]
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Downtown Toronto,[Queen's Park]


In [562]:
#Remove all "Not Assigned" items from all lists
toronto['Neighbourhood'] = toronto.apply(lambda x: [item for item in x.Neighbourhood if item != 'Not assigned'], axis=1)

In [563]:
#Define a function to replace with Borough if a list is empty
def ReplaceNA(x):
    if len(x.Neighbourhood) == 0:
        return x.Borough
    else:
        return x.Neighbourhood

In [564]:
#var = []
#tester = ReplaceNA(var)
#tester
len(toronto.iloc[5][2])

0

In [565]:
#Replace empty lists with Buroughs
toronto['Neighbourhood'] = toronto.apply(lambda x: ReplaceNA(x), axis=1)    

In [566]:
toronto.head(6)

,Postcode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,[Harbourfront]
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Downtown Toronto,[Queen's Park]
5,M9A,Queen's Park,Queen's Park


## Toronto Neighbourhoods Dataframe Shape

In [574]:
toronto.shape

(103, 3)

# Geocode Locations

In [575]:
#Define a geocoding function
geolocator = Nominatim(user_agent='toronto_explorer')

#Get Latitude
def getLat(row, city, province, geolocator):
    postal = row
    geo = geolocator.geocode('{}, {}, {}'.format(postal, city, province))
    try:
        latitude = geo.latitude
    except:
        return np.nan
    return latitude

#Get Longitude
def getLong(row, city, province, geolocator):
    postal = row
    geo = geolocator.geocode('{}, {}, {}'.format(postal, city, province))
    try:
        longitude = geo.longitude
    except:
        return np.nan
    return longitude

In [576]:
#latitude = getLat('M1B', 'Toronto', 'Ontario', geolocator)
#print(latitude)

In [577]:
#Obtain lat/long
geolocator = Nominatim(user_agent='toronto_explorer')
city = 'Toronto'
province = 'Ontario'
toronto['Latitude'] = toronto['Postcode'].apply(lambda row: getLat(row, city, province, geolocator))
toronto['Longitude'] = toronto['Postcode'].apply(lambda row: getLong(row, city, province, geolocator))

In [578]:
toronto.head(6)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.653963,-79.387207
1,M4A,North York,[Victoria Village],NaN,NaN
2,M5A,Downtown Toronto,[Harbourfront],NaN,NaN
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",NaN,NaN
4,M7A,Downtown Toronto,[Queen's Park],43.653963,-79.387207
5,M9A,Queen's Park,Queen's Park,NaN,NaN


In [579]:
#Check NA Values
toronto.isna().sum()

Postcode          0
Borough           0
Neighbourhood     0
Latitude         76
Longitude        76
dtype: int64

In [580]:
#Too many NA values, so upload CSV and merge
latlng = pd.read_csv('Toronto_Geospatial_Coordinates.csv')
latlng.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [581]:
toronto_2 = toronto.copy()
toronto_2.drop(columns=['Latitude', 'Longitude'], inplace=True)

toronto_merge = toronto_2.merge(latlng,
                               how='left',
                               left_on='Postcode',
                               right_on='Postcode')

In [582]:
toronto_merge.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,[Harbourfront],43.654260,-79.360636
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
4,M7A,Downtown Toronto,[Queen's Park],43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
7,M3B,North York,[Don Mills North],43.745906,-79.352188
8,M4B,East York,"[Woodbine Gardens, Parkview Hill]",43.706397,-79.309937
9,M5B,Downtown Toronto,"[Ryerson, Garden District]",43.657162,-79.378937


# K-Means Clustering Analysis

## Create Toronto Only Data

In [583]:
toronto = toronto_merge.copy()

In [654]:
#Filter out any row that doesn't conatin 'Toronto' as a substring
toronto_data = toronto[toronto.Borough.str.contains('Toronto')]
toronto_data['Neighbourhood'] = toronto_data['Neighbourhood'].apply(lambda x: tuple(x))
toronto_data.head()

C:\Users\quinlan.anderson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"(Harbourfront,)",43.654260,-79.360636
4,M7A,Downtown Toronto,"(Queen's Park,)",43.662301,-79.389494
9,M5B,Downtown Toronto,"(Ryerson, Garden District)",43.657162,-79.378937
15,M5C,Downtown Toronto,"(St. James Town,)",43.651494,-79.375418
19,M4E,East Toronto,"(The Beaches,)",43.676357,-79.293031


In [596]:
toronto_data.shape

(39, 5)

## Foursquare Credentialing and Setup

In [600]:
CLIENT_ID = 'OJRQECM1MUHF3FRN45B11S1QTHHT2G3W15BOBMZTM3NN3QI4' # your Foursquare ID
CLIENT_SECRET = 'DMT4UAWKWY3WGL4QEVOCBGE4MVODEOJTYWPZDYAUDF1X4ST0' # your Foursquare Secret
VERSION = '20180605'
#For 100 Venue Limit
LIMIT = 100
#For 500 Meter Radius
RADIUS = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJRQECM1MUHF3FRN45B11S1QTHHT2G3W15BOBMZTM3NN3QI4
CLIENT_SECRET:DMT4UAWKWY3WGL4QEVOCBGE4MVODEOJTYWPZDYAUDF1X4ST0


## Get Nearby Venues and Append to Toronto Dataframe

In [613]:
#Get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [614]:
#Apply the f(x) to get results
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                latitudes=toronto_data['Latitude'],
                                longitudes=toronto_data['Longitude']
                                )

In [615]:
print(toronto_venues.shape)
toronto_venues.head()

(1717, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[Harbourfront],43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,[Harbourfront],43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,[Harbourfront],43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,[Harbourfront],43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,[Harbourfront],43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [633]:
'Neighborhood' in toronto_venues['Venue Category']

False

In [620]:
#Convert Neighborhood list to a tuple
toronto_venues['Neighborhood'] = toronto_venues['Neighborhood'].apply(lambda x: tuple(x))


In [621]:
#Check the number of venues returned by each neighborhood grouping
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"(Adelaide, King, Richmond)",100,100,100,100,100,100
"(Berczy Park,)",58,58,58,58,58,58
"(Brockton, Exhibition Place, Parkdale Village)",24,24,24,24,24,24
"(Business Reply Mail Processing Centre 969 Eastern,)",18,18,18,18,18,18
"(CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara)",14,14,14,14,14,14
"(Cabbagetown, St. James Town)",48,48,48,48,48,48
"(Central Bay Street,)",85,85,85,85,85,85
"(Chinatown, Grange Park, Kensington Market)",86,86,86,86,86,86
"(Christie,)",18,18,18,18,18,18


In [639]:
#Count the unique values
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues['Venue Category'].unique()

There are 229 unique categories


array(['Bakery', 'Coffee Shop', 'Gym / Fitness Center', 'Spa',
       'Restaurant', 'Breakfast Spot', 'Pub', 'Park', 'Historic Site',
       'Chocolate Shop', 'Farmers Market', 'Performing Arts Venue',
       'Dessert Shop', 'French Restaurant', 'Café', 'Mexican Restaurant',
       'Event Space', 'Yoga Studio', 'Asian Restaurant', 'Ice Cream Shop',
       'Shoe Store', 'Theater', 'Art Gallery', 'Brewery',
       'Electronics Store', 'Beer Store', 'Bank', 'Hotel',
       'Health Food Store', 'Antique Shop', 'Portuguese Restaurant',
       'Italian Restaurant', 'Gym', 'Creperie', 'Burrito Place',
       'Beer Bar', 'Arts & Crafts Store', 'Sushi Restaurant',
       'Hobby Shop', 'Fried Chicken Joint', 'Diner', 'Burger Joint',
       'Nightclub', 'Seafood Restaurant', 'Chinese Restaurant',
       'Fast Food Restaurant', 'Juice Bar', 'Sandwich Place',
       'College Auditorium', 'Bar', 'Clothing Store', 'Comic Shop',
       'Pizza Place', 'Plaza', 'Tea Room', 'Music Venue',
       'Ramen R

## Analyzing All Neighborhoods

In [641]:
# one hot encoding
toronto_onehot = []
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"(Harbourfront,)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(Harbourfront,)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [643]:
toronto_onehot.shape

(1717, 230)

## Take the mean of the frequency of occurence for each category

In [645]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [646]:
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"(Adelaide, King, Richmond)",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.010000,0.0,0.03,0.000000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.030000,0.000000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.01,0.000000,0.070000,0.0,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.01,0.01,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0

In [647]:
# See New Size

toronto_grouped.shape

(39, 230)

## Top 5 Common Venues for Each Venue

In [648]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [649]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"(Adelaide, King, Richmond)",Coffee Shop,Café,Steakhouse,Bar,Breakfast Spot,Hotel,Restaurant,Asian Restaurant,Thai Restaurant,Cosmetics Shop
1,"(Berczy Park,)",Coffee Shop,Cocktail Bar,Cheese Shop,Steakhouse,Café,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Bistro
2,"(Brockton, Exhibition Place, Parkdale Village)",Nightclub,Coffee Shop,Breakfast Spot,Café,Bakery,Convenience Store,Italian Restaurant,Stadium,Intersection,Restaurant
3,(Business Reply Mail Processing Centre 969 Eas...,Yoga Studio,Fast Food Restaurant,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"(CN Tower, Bathurst Quay, Island airport, Harb...",Airport Terminal,Airport Lounge,Boutique,Bar,Coffee Shop,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Service,Airport Gate


## Cluster Neighborhoods

In [651]:
toronto_grouped.drop('Neighbourhood', 1)

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.030000,0.000000,0.000000,0.01,0.020000,0.010000,0.0,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.00000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0000

In [653]:
#Set number of clusters

k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0])

DF That includes cluster grouping # and top 10 venues

In [658]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"(Harbourfront,)",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Café,Yoga Studio,Farmers Market,Restaurant
4,M7A,Downtown Toronto,"(Queen's Park,)",43.662301,-79.389494,0,Coffee Shop,Park,Gym,Yoga Studio,Sandwich Place,Beer Bar,Italian Restaurant,Fast Food Restaurant,Juice Bar,Seafood Restaurant
9,M5B,Downtown Toronto,"(Ryerson, Garden District)",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Fast Food Restaurant,Plaza,Italian Restaurant
15,M5C,Downtown Toronto,"(St. James Town,)",43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Clothing Store,Hotel,Beer Bar
19,M4E,East Toronto,"(The Beaches,)",43.676357,-79.293031,3,Pub,Park,Neighborhood,Health Food Store,Trail,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [660]:
#Toronto Address

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [662]:
# create map


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1:

In [663]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Café,Yoga Studio,Farmers Market,Restaurant
4,Downtown Toronto,0,Coffee Shop,Park,Gym,Yoga Studio,Sandwich Place,Beer Bar,Italian Restaurant,Fast Food Restaurant,Juice Bar,Seafood Restaurant
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Fast Food Restaurant,Plaza,Italian Restaurant
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Clothing Store,Hotel,Beer Bar
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Cheese Shop,Steakhouse,Café,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Bistro
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Japanese Restaurant,Juice Bar,Burger Joint,Spa,Dessert Shop
25,Downtown Toronto,0,Grocery Store,Café,Park,Gas Station,Italian Restaurant,Candy Store,Bank,Diner,Restaurant,Baby Store
30,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Breakfast Spot,Hotel,Restaurant,Asian Restaurant,Thai Restaurant,Cosmetics Shop
31,West Toronto,0,Pharmacy,Bakery,Grocery Store,Middle Eastern Restaurant,Music Venue,Park,Recording Studio,Café,Brewery,Supermarket
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,History Museum


### Cluster 2

In [664]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 3

In [665]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,2,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 4

In [666]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,3,Pub,Park,Neighborhood,Health Food Store,Trail,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
91,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Cluster 5

In [667]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Central Toronto,4,Gym,Breakfast Spot,Park,Food & Drink Shop,Hotel,Department Store,Dog Run,Sandwich Place,Discount Store,Event Space
